In [7]:
import os
import re
import pandas as pd


INPUT_PATH = "院校数据.xlsx"
OUTPUT_PATH = "本科院校.csv"
SHEET_NAME = None

def normalize_col(s: str) -> str:
    return re.sub(r"\s+", "", str(s or "")).strip()


def find_column(df_cols, candidates):
    norm_cols = {normalize_col(c): c for c in df_cols}
    for cand in candidates:
        cand_norm = normalize_col(cand)
        if cand_norm in norm_cols:
            return norm_cols[cand_norm]
    return None


def last5_digits(code) -> str:
    s = str(code or "").strip()
    digits = re.sub(r"\D", "", s)
    return digits[-5:] if len(digits) >= 5 else digits


def main():
    if not os.path.exists(INPUT_PATH):
        raise FileNotFoundError(f"输入文件不存在：{INPUT_PATH}")

    if SHEET_NAME is None:
        df = pd.read_excel(INPUT_PATH, dtype=str)
    else:
        df = pd.read_excel(INPUT_PATH, sheet_name=SHEET_NAME, dtype=str)

    name_col = find_column(df.columns, ["学校名称"])
    code_col = find_column(df.columns, ["学校标识码"])
    location_col = find_column(df.columns, ["所在地"])
    level_col = find_column(df.columns, ["办学层次"])

    missing = [c for c, v in {
        "学校名称": name_col, "学校标识码": code_col, "所在地": location_col, "办学层次": level_col
    }.items() if v is None]
    if missing:
        raise KeyError(f"缺少必要列：{', '.join(missing)}；当前表头：{list(df.columns)}")

    # 生成后五位与层次标准化列
    df["_标识码后五位"] = df[code_col].apply(last5_digits)
    df["_层次_标准化"] = df[level_col].astype(str).str.strip()

    # 过滤包含“本科”的记录（包含匹配更稳健）
    mask_undergrad = df["_层次_标准化"].str.contains("本科", na=False)
    result = df[mask_undergrad][[name_col, "_标识码后五位", location_col, level_col]].copy()
    result.columns = ["学校", "全国统一招生代码", "所在地", "办学层次"]

    # 导出到 CSV 或 Excel
    out_dir = os.path.dirname(OUTPUT_PATH)
    if out_dir:
        os.makedirs(out_dir, exist_ok=True)

    if OUTPUT_PATH.lower().endswith(".xlsx"):
        result.to_excel(OUTPUT_PATH, index=False)
    else:
        result.to_csv(OUTPUT_PATH, index=False, encoding="utf-8-sig")

    col = "全国统一招生代码"
    codes = result[col].fillna("").astype(str).str.strip()
    nonempty_mask = codes != ""
    total = int(nonempty_mask.sum())
    distinct = int(codes[nonempty_mask].nunique())
    dup_mask = nonempty_mask & codes.duplicated(keep=False)
    dups = result[dup_mask].copy()

    print(f"记录数（非空代码）：{total}")
    print(f"去重后代码数：{distinct}")
    print(f"重复代码记录数：{len(dups)}")

    if not dups.empty:
        stats = (
            dups.assign(
                _code_norm=codes[dup_mask],
                _name=result["学校"][dup_mask].astype(str).str.strip()
            )
            .groupby("_code_norm")
            .agg(
                数量=("全国统一招生代码", "count"),
                学校清单=("_name", lambda s: "、".join(sorted(set(s))))
            )
            .sort_values("数量", ascending=False)
        )
        print("\n重复代码统计：")
        print(stats)

        dup_out_name = os.path.splitext(os.path.basename(OUTPUT_PATH))[0] + "_重复代码.csv"
        dup_out = os.path.join(out_dir if out_dir else "", dup_out_name)
        dups.to_csv(dup_out, index=False, encoding="utf-8-sig")
        print(f"\n已导出重复明细：{dup_out}")
    else:
        print("无重复代码。")

if __name__ == "__main__":
    main()


记录数（非空代码）：1365
去重后代码数：1365
重复代码记录数：0
无重复代码。
